In [1]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging
import numpy as np
import plotly.graph_objects as go
from PIL import Image

import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.autograd import Variable

from torch.optim import Adam, SGD
import gc

from ResNet_CNN import *

import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision
from functools import partial
from collections import OrderedDict

import torchvision.models as models

from torchsummary import summary

In [192]:
dta = pd.read_csv('SP500.csv')

In [193]:
dta['Date'] = pd.to_datetime(dta['Date'],format = '%Y-%m-%d')

In [194]:
dta = dta.set_index(dta['Date'])

In [195]:
dta.drop(['Date'], axis=1, inplace=True)

In [210]:
for tick in dta.columns:
    tick_series = dta[tick]
    start_pos = tick_series.first_valid_index()
    valid_series = tick_series.loc[start_pos:]
    

,NKE,DDS,IPG,LLY,MTG,NAVI,WY,CI,PLU,WTR,...,VAL,EMR,WU,AVP,TEX,XOM,TPR,TER,BLK,SPY
Date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,0.14,40.78,6.37,4.05,11.29,NaN,17.35,3.42,NaN,0.17,...,7.9348,3.64,NaN,1.45,4.37,3.48,NaN,3.42,NaN,26.30
1993-02-01,0.14,40.26,6.42,4.01,11.56,NaN,17.56,3.55,NaN,0.17,...,7.5570,3.64,NaN,1.49,4.55,3.50,NaN,3.27,NaN,26.49
1993-02-02,0.14,40.68,6.19,3.96,11.67,NaN,17.62,3.54,NaN,0.17,...,7.9348,3.62,NaN,1.50,4.66,3.54,NaN,3.24,NaN,26.54
1993-02-03,0.14,42.45,6.47,4.04,11.81,NaN,17.56,3.66,NaN,0.18,...,7.9348,3.69,NaN,1.52,4.61,3.56,NaN,3.45,NaN,26.82
1993-02-04,0.15,42.45,6.47,4.03,12.03,NaN,17.83,3.73,NaN,0.18,...,7.5570,3.78,NaN,1.52,4.78,3.70,NaN,3.51,NaN,26.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-08,90.46,28.26,17.32,152.79,7.13,7.77,19.85,189.27,NaN,NaN,...,0.3900,55.56,19.93,NaN,14.51,45.30,14.88,65.09,498.78,292.44
2020-05-11,90.93,26.96,16.69,157.81,7.14,7.35,19.35,188.99,NaN,NaN,...,0.3900,55.48,19.31,NaN,13.81,44.87,13.92,63.11,493.11,292.50
2020-05-12,88.26,25.22,16.13,156.98,6.73,7.04,18.20,189.56,NaN,NaN,...,0.3700,52.93,18.87,NaN,12.98,44.12,13.19,61.30,454.44,286.67


In [211]:
dta.SPY.first_valid_index()

Timestamp('1993-01-29 00:00:00')

In [216]:
(dta.SPY.loc[dta.SPY.first_valid_index():]).isna()

Date
1993-01-29    False
1993-02-01    False
1993-02-02    False
1993-02-03    False
1993-02-04    False
              ...  
2020-05-08    False
2020-05-11    False
2020-05-12    False
2020-05-13    False
2020-05-14    False
Name: SPY, Length: 6892, dtype: bool

In [219]:
dta.isna().sum(axis=0)

NKE      19
DDS      19
IPG      19
LLY      19
MTG      19
       ... 
XOM      19
TPR    1961
TER      19
BLK    1704
SPY      19
Length: 574, dtype: int64

In [197]:
dta = dta[dta.index >= datetime(1993, 1, 29)]
dta.drop([datetime(1995, 9, 4), datetime(2020, 5, 15), datetime(1995, 11, 23)], axis=0, inplace=True)

In [202]:
dta.drop(['PA'], axis=1, inplace=True)

In [203]:
ticker_universe = list(set(dta.columns))

In [204]:
ticker_universe.remove('SPY')

In [205]:
len(ticker_universe)

573

In [26]:
sub_dta = dta.iloc[20:40]

In [48]:
next_dta = dta.iloc[40:60]

In [45]:
ranking = sub_dta.apply(lambda x: (x[-1] - x[0])/x[0])

select_tic = ranking.dropna().sort_values()[:10].index.to_list()

In [50]:
select_tic

['IAC', 'WDC', 'BSX', 'NAV', 'EPHC', 'GILD', 'BIIB', 'MNST', 'MRK', 'LLY']

In [92]:
(np.array(next_dta_tic) @ weight)[-1]

49257.619999999995

In [108]:
test = np.array([])

test = np.append(test, np.array(next_dta_tic) @ weight)

test

array([187889.3477, 156855.5221, 172021.1975, 173811.567 , 159781.487 ,
       151719.097 , 113299.297 , 113913.0974, 105514.744 , 102994.027 ,
       187843.437 , 129839.087 , 130301.347 , 110537.607 , 124086.647 ,
       123024.232 , 118299.749 , 129731.934 , 124741.474 , 127846.9895])

In [109]:
total_eq // 10

C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning:

invalid value encountered in double_scalars



nan

In [110]:
5000 / np.array(next_dta_tic.iloc[0])

array([2.17391304e+04, 8.92857143e+01, 5.36480687e+02, 1.63934426e+03,
       1.00000000e+07, 1.35501355e+03, 1.08932462e+03, 3.54609929e+03,
       1.49253731e+03, 2.24346031e+03])

In [111]:
weight = (5000 / np.array(next_dta_tic.iloc[0])).round()

In [112]:
weight

array([2.1739e+04, 8.9000e+01, 5.3600e+02, 1.6390e+03, 1.0000e+07,
       1.3550e+03, 1.0890e+03, 3.5460e+03, 1.4930e+03, 2.2430e+03])

In [113]:
next_dta_tic = next_dta[select_tic]

In [114]:
sub_dta

,NKE,DDS,IPG,LLY,MTG,NAVI,WY,CI,PLU,WTR,...,VAL,EMR,WU,AVP,TEX,XOM,TPR,TER,BLK,SPY
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-30,97.92,67.26,22.93,141.27,13.72,14.69,29.71,196.95,NaN,52.11,...,5.20,72.63,27.32,NaN,25.75,62.66,26.27,68.75,535.63,325.76
2020-01-31,96.03,60.49,22.43,138.27,13.62,14.17,28.60,192.34,NaN,51.71,...,5.11,70.47,26.63,NaN,25.21,60.08,25.41,65.89,523.38,319.85
2020-02-03,99.00,60.28,22.51,139.62,13.93,13.81,27.88,191.73,NaN,53.49,...,5.12,71.42,26.81,NaN,25.33,58.73,25.67,67.70,526.09,322.23
2020-02-04,101.10,60.37,22.60,142.03,14.09,13.62,27.84,196.82,NaN,52.76,...,5.27,72.05,27.18,NaN,25.76,58.00,26.82,70.37,531.21,327.14
2020-02-05,100.26,62.78,23.24,145.90,14.34,13.87,28.57,206.44,NaN,53.10,...,5.62,74.78,27.64,NaN,26.97,60.67,28.00,71.18,539.39,330.91
2020-02-06,99.99,64.05,22.94,145.85,14.19,13.90,28.31,211.82,NaN,52.97,...,5.32,74.03,27.69,NaN,26.65,59.84,28.60,71.27,550.49,332.03
2020-02-07,99.16,64.44,22.75,144.97,14.08,13.81,28.24,208.88,NaN,52.99,...,5.23,72.57,27.37,NaN,25.42,59.45,27.58,69.35,551.56,330.26
2020-02-10,99.74,64.75,22.89,144.08,14.05,13.88,28.31,210.28,NaN,53.49,...,4.76,72.20,28.01,NaN,25.52,58.82,28.11,70.45,556.87,332.72
2020-02-11,99.74,66.46,22.98,142.83,14.07,14.29,28.55,215.14,NaN,53.04,...,5.07,72.63,27.95,NaN,26.10,59.38,28.18,72.29,566.14,333.30


In [208]:
cycle = 20
N_portfolio = 10
total_eq = 5e4

portfolio_hist = np.array([total_eq])

T = dta.shape[0] // cycle
T

344

In [209]:
for t in range(102, T-1):
    sub_dta = dta.iloc[(t-1)*cycle : t*cycle]
    ranking = sub_dta.apply(lambda x: (x[-1] - x[0])/x[0])
    select_tic = ranking.dropna().sort_values()[:10].index.to_list()
    print(select_tic)
    next_dta = dta.iloc[t*cycle : (t+1)*cycle][select_tic]
    
    eq = total_eq // N_portfolio
    weight = (eq / np.array(next_dta.iloc[0])).round()
    print(weight)
    temp_hist = np.array(next_dta) @ weight
    print(list(temp_hist))
    total_eq = temp_hist[-1]
    
    # portfolio_hist = np.append(portfolio_hist, temp_hist)
    
    print("{}/{}".format(t, T-1))

['ACR', 'CD', 'JNPR', 'ALXN', 'NTAP', 'GLW', 'ORCL', 'ADBE', 'VRSN', 'CSCO']
[25000.   222.    71.   543.   147.   165.   256.   271.   102.   229.]
[49993.100000000006, 52595.03, 53574.38999999999, 49746.43, 47764.829999999994, 46478.44, 45118.82, 45923.030000000006, 47436.67, 43109.8, 41472.950000000004, 42889.10999999999, 40215.0, 40493.340000000004, 41415.88, 40528.189999999995, 39061.93, 36801.41, 34414.159999999996, 36767.24999999999]
102/343
['AKAM', 'FFIV', 'ALXN', 'NTAP', 'VRTX', 'CTXS', 'AABA', 'CMT', 'ETFC', 'GLW']
[ 327. 1250.  735.  203.  106.  272. 1759. 4223.   56.  199.]
[36782.0592, 35329.2999, 33751.6499, 37356.7799, 35684.289899999996, 34482.4899, 36677.4799, 37901.12989999999, 37181.4852, 37619.205200000004, 34746.845199999996, 34058.7699, 35374.5599, 32786.859899999996, 29006.6083, 28439.946699999997, 32666.9414, 32761.1446, 33765.8446, 36782.686100000006]
103/343
['ISRG', 'EPHC', 'DXC', 'PCG', 'JNPR', 'HUM', 'VIAV', 'SANM', 'INTU', 'NTAP']
[1649. 1042.  394.  705.

['NSP', 'CGCO', 'SEE', 'CCI', 'GLW', 'AMT', 'LOM', 'NVDA', 'PKI', 'MXM']
[ 8689. 59245.  1075.  4688.  5214.  3998.  3017.  1680.  1448.  4073.]
[65166.68000000001, nan, nan, nan, nan, nan, 68910.58, nan, 65284.100000000006, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
121/343
['MCHP', 'LOM', 'AMD', 'LRCX', 'MXIM', 'FFIV', 'TER', 'DLTR', 'MU', 'TEN']


C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in double_scalars



[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
122/343
['AMT', 'AES', 'FLS', 'SANM', 'URI', 'AMD', 'NFLX', 'MLSS', 'CLF', 'LH']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
123/343
['THC', 'CI', 'TEX', 'HRB', 'IPG', 'GT', 'UHS', 'HES', 'DRI', 'UW']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
124/343
['SPC', 'LOM', 'MANT', 'BIIB', 'PDCO', 'CU', 'WMB', 'SRCL', 'TEN', 'PEAK']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
125/343
['GME', 'MU', 'UW', 'ACR', 'HD', 'EA', 'URBN', 'ISRG', 'WAT', 'MLSS']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

158/343
['PAR', 'KNE', 'STA', 'CMT', 'ANF', 'IAC', 'FOSL', 'MANT', 'WYNN', 'CTB']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
159/343
['STA', 'CGCO', 'ANF', 'OI', 'PAR', 'STX', 'LEG', 'AVP', 'MLSS', 'ZBH']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
160/343
['FLIR', 'CMT', 'NAV', 'X', 'XLNX', 'CF', 'NBL', 'ACR', 'APA', 'OXY']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
161/343
['PPE', 'IIN', 'THC', 'SPC', 'ACAD', 'CHV', 'STA', 'MLSS', 'MXIM', 'CNX']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
162/343
['ALXN', 'HMT', 'NB', 'PDCO', 'URBN', 'PIR', 'CHV', 'LOM', 'MLSS', 'CERN']
[nan nan nan nan nan na

...

C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning:

invalid value encountered in double_scalars



['NOB', 'SNS', 'TERX', 'REI', 'DNR', 'WPX', 'PLU', 'RIG', 'NBR', 'FOH']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
276/343
['NOB', 'PP', 'X', 'URI', 'SWN', 'NB', 'AVP', 'CZR', 'QEP', 'CNX']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
277/343


C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in double_scalars



['NOB', 'PPE', 'ACR', 'TERX', 'MEE', 'HMT', 'SPC', 'SRL', 'PP', 'LOM']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
278/343
['NOB', 'SPC', 'ACR', 'PIR', 'SCE', 'CHK', 'VAL', 'PE', 'MXM', 'ANF']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
279/343
['TERX', 'SCE', 'PP', 'MEE', 'CLF', 'ACAD', 'CMX', 'FOH', 'LOM', 'FTN']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
280/343
['SPC', 'ACR', 'COFI', 'AHC', 'CMX', 'MAY', 'STA', 'CZR', 'MANT', 'AMD']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
281/343
['SPC', 'PP', 'NHLI', 'LMC', 'CD', 'KNE', 'FTR', 'PPE', 'CZR', 'SCE']
[nan nan nan nan nan nan nan nan nan nan

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
314/343
['ACR', 'FTN', 'FTR', 'PPE', 'CDIC', 'NOB', 'PIR', 'KNE', 'LB', 'PCG']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
315/343
['NHLI', 'SWN', 'SANM', 'CHK', 'PE', 'PIR', 'MNK', 'VAL', 'AKS', 'QEP']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
316/343
['PDCO', 'MTW', 'NFX', 'ODP', 'DISCB', 'NAV', 'DLTR', 'DVN', 'GT', 'WMT']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
317/343
['ABBV', 'MGI', 'X', 'MAT', 'BIG', 'MLSS', 'CD', 'AMD', 'AKS', 'PBI']
[nan nan nan nan nan nan nan nan nan nan]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
318/343


In [207]:
t = 103

sub_dta = dta.iloc[(t-1)*cycle : t*cycle]
ranking = sub_dta.apply(lambda x: (x[-1] - x[0])/x[0])
select_tic = ranking.dropna().sort_values()[:10].index.to_list()
print(select_tic)
next_dta = dta.iloc[t*cycle : (t+1)*cycle][select_tic]

eq = total_eq // N_portfolio
weight = (eq / np.array(next_dta.iloc[0])).round()
print(weight)
temp_hist = np.array(next_dta) @ weight
print(list(temp_hist))
total_eq = temp_hist[-1]

['AKAM', 'FFIV', 'ALXN', 'NTAP', 'VRTX', 'CTXS', 'AABA', 'CMT', 'ETFC', 'GLW']
[ 444. 1701. 1000.  276.  144.  370. 2392. 5744.   76.  271.]
[50010.717599999996, 48036.247200000005, 45891.877199999995, 50793.8372, 48520.5972, 46887.4972, 49873.4872, 51537.5772, 50556.88560000001, 51152.44560000001, 47246.0056, 46309.9172, 48098.36719999999, 44581.2972, 39441.4324, 38670.6076, 44417.279200000004, 44545.9288, 45912.1788, 50012.4008]


In [185]:
sub_dta = dta.iloc[(t-1)*cycle : t*cycle]
ranking = sub_dta.apply(lambda x: (x[-1] - x[0])/x[0])

In [188]:
select_tic

['AKAM', 'PA', 'FFIV', 'ALXN', 'NTAP', 'VRTX', 'CTXS', 'AABA', 'CMT', 'ETFC']

In [189]:
next_dta = dta.iloc[t*cycle : (t+1)*cycle][select_tic]

In [190]:
next_dta

,AKAM,PA,FFIV,ALXN,NTAP,VRTX,CTXS,AABA,CMT,ETFC
Date,,,,,,,,,,
2001-03-14,11.25,NaN,2.94,5.00,18.12,34.62,13.51,2.09,0.8704,66.16
2001-03-15,9.31,0.01,2.78,5.12,16.03,33.75,14.04,2.04,0.7913,67.94
2001-03-16,8.22,0.01,2.52,5.02,16.19,31.62,14.09,1.85,0.7913,64.38
2001-03-19,9.47,0.01,3.25,5.69,17.48,34.31,14.82,2.04,0.7913,71.58
2001-03-20,8.88,0.01,3.38,4.97,15.93,32.06,15.46,1.90,0.7913,66.25
2001-03-21,8.34,0.01,3.24,5.05,16.46,29.94,13.80,1.87,0.7913,60.83
2001-03-22,8.69,0.01,3.36,6.19,19.23,29.19,15.26,2.03,0.7913,58.43
2001-03-23,8.53,0.01,3.72,6.80,18.06,31.12,16.82,1.97,0.7913,61.81
2001-03-26,9.41,0.01,3.28,6.34,17.00,34.88,17.07,1.93,0.6924,63.23
